# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770835541141                   -0.52    9.0         
  2   -2.772141834692       -2.88       -1.32    1.0    152ms
  3   -2.772170305099       -4.55       -2.52    1.0    132ms
  4   -2.772170658289       -6.45       -3.32    1.0    134ms
  5   -2.772170719697       -7.21       -3.77    2.0    169ms
  6   -2.772170722948       -8.49       -4.89    1.0    137ms
  7   -2.772170723005      -10.25       -5.04    2.0    157ms
  8   -2.772170723015      -11.02       -6.27    1.0    162ms
  9   -2.772170723015      -12.77       -6.74    2.0    162ms
 10   -2.772170723015   +  -15.35       -7.26    1.0    167ms
 11   -2.772170723015   +  -14.40       -8.01    2.0    173ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770721077634                   -0.53    9.0         
  2   -2.7

1.7735578867439137

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770697326927                   -0.53    8.0         
  2   -2.772052710749       -2.87       -1.31    1.0    130ms
  3   -2.772082939142       -4.52       -2.55    1.0    133ms
  4   -2.772083379823       -6.36       -3.58    1.0    171ms
  5   -2.772083416264       -7.44       -3.93    2.0    152ms
  6   -2.772083417744       -8.83       -5.00    1.0    135ms
  7   -2.772083417805      -10.22       -5.20    2.0    161ms
  8   -2.772083417810      -11.33       -5.73    1.0    164ms
  9   -2.772083417811      -12.14       -6.80    2.0    168ms
 10   -2.772083417811      -14.65       -7.40    1.0    179ms
 11   -2.772083417811   +  -14.65       -8.27    1.0    143ms

Polarizability via ForwardDiff:       1.7725349665480812
Polarizability via finite difference: 1.7735578867439137
